---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.metrics import roc_auc_score

def blight_model():
    
    # Your code here
    data = pd.read_csv("train.csv", encoding = 'ISO-8859-1',low_memory=False)
    addrs = pd.read_csv("addresses.csv", engine='python') # encoding = 'ISO-8859-1',low_memory=False)
    latlons = pd.read_csv("latlons.csv", engine='python') # encoding = 'ISO-8859-1',low_memory=False)
    test = pd.read_csv("test.csv", engine='python') # encoding = 'ISO-8859-1',low_memory=False)
    
    addrs = addrs.merge(latlons, left_on='address', right_on='address')
    data = data.merge(addrs[['ticket_id', 'lat', 'lon']], left_on='ticket_id', right_on='ticket_id')
    
    data = data[~data.compliance.isnull()]
    data = data[data['agency_name'] != 'Neighborhood City Halls']
    data = data[data['city'] == 'Detroit']
    data['ticket_issued_date'] = pd.to_datetime(data['ticket_issued_date'])
    data = data[~data.hearing_date.isnull()]
    data['hearing_date'] = pd.to_datetime(data['hearing_date'])
    data = data[data['ticket_issued_date'].apply(lambda x: x.year >= 2004 and x.year <= 2011)]
    data['date_diff'] = data['hearing_date'] - data['ticket_issued_date']
    data['date_diff'] = data['date_diff'].dt.days
    data['violation_code'] = data['violation_code'].apply(lambda x: x[:2])
    
    drop_cols = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', \
                 'compliance_detail', 'discount_amount', 'violation_zip_code', 'non_us_str_code', 'zip_code', \
                 'grafitti_status', 'inspector_name', 'violator_name', 'violation_description', 'state_fee',\
                 'admin_fee', 'clean_up_cost', 'ticket_issued_date', 'hearing_date', 'violation_street_name',\
                 'state', 'country', 'mailing_address_str_name', 'mailing_address_str_number', 'city']
    data.drop(drop_cols, axis=1, inplace=True)
    
    le = LabelEncoder()
    le.fit(data['agency_name'].astype(str))
    data['agency_name'] = le.transform(data['agency_name'].astype(str))
    
    le.fit(data['disposition'].astype(str))
    data['disposition'] = le.transform(data['disposition'].astype(str))
    
    le.fit(data['violation_code'].astype(str))
    data['violation_code'] = le.transform(data['violation_code'].astype(str))
    
    test = test.merge(addrs[['ticket_id', 'lat', 'lon']], left_on='ticket_id', right_on='ticket_id')
    
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
    test.hearing_date.fillna(method='pad', inplace=True)
    test.lat.fillna(method='pad', inplace=True)
    test.lon.fillna(method='pad', inplace=True)
    test['hearing_date'] = pd.to_datetime(test['hearing_date'])        
    test['date_diff'] = test['hearing_date'] - test['ticket_issued_date']
    test['date_diff'] = test['date_diff'].dt.days
    test['violation_code'] = test['violation_code'].apply(lambda x: x[:2])
    
    drop_col = ['discount_amount', 'violation_zip_code', 'non_us_str_code', 'grafitti_status', 'zip_code',\
                 'inspector_name', 'violator_name', 'violation_description', 'state_fee', 'admin_fee',\
                 'clean_up_cost', 'ticket_issued_date', 'hearing_date', 'violation_street_name',\
                 'state', 'country', 'mailing_address_str_name', 'mailing_address_str_number', 'city']
    test.drop(drop_col, axis=1, inplace=True)
    
    le.fit(test['agency_name'].astype(str))
    test['agency_name'] = le.transform(test['agency_name'].astype(str))
    
    le.fit(test['disposition'].astype(str))
    test['disposition'] = le.transform(test['disposition'].astype(str))
    
    le.fit(test['violation_code'].astype(str))
    test['violation_code'] = le.transform(test['violation_code'].astype(str))
    
    #print(test.shape)
    
    X = data.drop('compliance', axis=1)
    y = pd.DataFrame(data['compliance'])
    
    imputer = Imputer(strategy='median')
    imputer.fit(X)
    X = imputer.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    gbclf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
    y_pred = gbclf.predict_proba(X_test)
    ytest = gbclf.predict_proba(test)
    
    #print("training score : ", gbclf.score(X_train, y_train))
    #print("test score : ", gbclf.score(X_test, y_test))
    #print("roc-auc-score : ", roc_auc_score(y_test, y_pred[:,1]))
    #print("test score on test.csv file : ", ytest)
    
    compliance = pd.Series(data=ytest[:,1], index=test.ticket_id)
    
    return compliance

blight_model()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ticket_id
284932    0.208855
285362    0.099568
285361    0.152737
285338    0.390754
285346    0.399718
285345    0.390754
285347    0.536754
285342    0.954020
285530    0.106622
284989    0.583441
285344    0.517945
285343    0.264346
285340    0.279280
285341    0.568067
285349    0.399718
285348    0.390754
284991    0.583441
285532    0.169722
285406    0.115074
285001    0.419140
285006    0.483836
285405    0.099568
285337    0.075161
285496    0.332550
285497    0.224197
285378    0.089067
285589    0.099568
285585    0.147947
285501    0.193010
285581    0.089067
            ...   
376367    0.129256
376366    0.181566
376362    0.155062
376363    0.164146
376365    0.129256
376364    0.181566
376228    0.197172
376265    0.135946
376286    0.885222
376320    0.244150
376314    0.307118
376327    0.829470
376385    0.976111
376435    0.751520
376370    0.855631
376434    0.300336
376459    0.228886
376478    0.140304
376473    0.347239
376484    0.270733
376482    0.122990
37